In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [2]:
import os
import random
import tensorflow as tf

seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.config.experimental.enable_op_determinism()

In [3]:
df = pd.read_csv('fnspid_prices_title_sentiment.csv')
df = df.set_index('date')
df.index = pd.to_datetime(df.index)
stocks_to_remove = ['MMM', 'SBUX', 'SIRI', 'SLB', 'SLV', 'EEM' , 'EWJ', 'GDX', 'GLD', 'QQQ', 'SLV', 'USO', 'XLF', 'XLK', 'XLP', 'XLU', 'XLY']
df = df[~df['Stock_symbol'].isin(stocks_to_remove)]

In [4]:
df['target_binary'] = (df['movement_percent'] >= 0).astype(int)
df['target_binary'] = df['target_binary'].shift(-1)
df.dropna(inplace=True)

In [5]:
def sequences(df: DataFrame, window_size: int, feature_cols: list, target: str):
    X, y = [], []

    features = df[feature_cols].to_numpy()
    y_vals = df[target].to_numpy()

    for i in range(len(df) - window_size):
        X.append(features[i:i + window_size])
        y.append(y_vals[i + window_size])

    return np.array(X), np.array(y)

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

window_size = 20
feature_cols = ['open_logret', 'high_logret', 'low_logret', 'close_logret', 'volume_logret']
target = 'target_binary'
l2_value = 1e-4
batch_size = 8
epochs = 50
learning_rate = 0.001

results = []

for symbol in df['Stock_symbol'].unique():
    print(f"\n{'='*40}\nTraining model for: {symbol}\n{'='*40}")

    df_symbol = df[df['Stock_symbol'] == symbol].copy()
    price_cols = ['open', 'high', 'low', 'close', 'volume']
    for col in price_cols:
      df_symbol[f'{col}_logret'] = np.log(df_symbol[col]) - np.log(df_symbol[col].shift(1))
    df_symbol.dropna(inplace=True)

    train = df_symbol.loc['2014-01-01':'2021-03-29']
    val = df_symbol.loc['2021-03-30':'2022-07-28']
    test = df_symbol.loc['2022-07-29':'2023-12-01']

    if len(train) < window_size * 2 or len(test) < window_size:
        print(f"Not enough data for {symbol}, skipping.")
        continue

    X_train, y_train = sequences(train, window_size=window_size, feature_cols=feature_cols, target=target)
    X_val, y_val = sequences(val, window_size=window_size, feature_cols=feature_cols, target=target)
    X_test, y_test = sequences(test, window_size=window_size, feature_cols=feature_cols, target=target)

    X_train = np.array(X_train); y_train = np.array(y_train)
    X_val = np.array(X_val); y_val = np.array(y_val)
    X_test = np.array(X_test); y_test = np.array(y_test)

    for X in [X_train, X_val, X_test]:
        np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0, copy=False)

    mean = np.mean(X_train, axis=(0,1))
    std = np.std(X_train, axis=(0,1))
    std[std == 0] = 1e-8

    # X_train_norm = (X_train - mean) / std
    # X_val_norm = (X_val - mean) / std
    # X_test_norm = (X_test - mean) / std

    unique, counts = np.unique(y_test, return_counts=True)
    dist = dict(zip(unique, counts))
    total = counts.sum()
    pct_0 = 100 * dist.get(0, 0) / total
    pct_1 = 100 * dist.get(1, 0) / total
    print(f"Test class distribution: 0s={pct_0:.2f}%, 1s={pct_1:.2f}%")

    model = Sequential([
        Input(shape=(window_size, len(feature_cols))),
        Conv1D(filters=16, kernel_size=3, strides=1, activation='relu', padding='same'),
        BatchNormalization(),
        Conv1D(filters=8, kernel_size=3, strides=1, activation='relu', padding='same'),
        BatchNormalization(),
        LSTM(16),
        Dropout(0.1),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_value))
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC', 'accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=0
    )

    y_val_pred_prob = model.predict(X_val)
    y_val_pred = (y_val_pred_prob > 0.5).astype(int)

    y_test_pred_prob = model.predict(X_test)
    y_test_pred = (y_test_pred_prob > 0.5).astype(int)

    val_auc = roc_auc_score(y_val, y_val_pred_prob)
    val_f1 = f1_score(y_val, y_val_pred)
    val_mcc = matthews_corrcoef(y_val, y_val_pred)

    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    test_f1 = f1_score(y_test, y_test_pred)
    test_mcc = matthews_corrcoef(y_test, y_test_pred)
    test_acc = np.mean(y_test_pred.flatten() == y_test.flatten())
    test_cm = confusion_matrix(y_test, y_test_pred)

    print(f"[{symbol}] Test AUC: {test_auc:.4f} | F1: {test_f1:.4f} | MCC: {test_mcc:.4f} | ACC: {test_acc:.4f}")
    print(f"Confusion matrix:\n{test_cm}\n")

    results.append({
        'Symbol': symbol,
        'Val_AUC': val_auc,
        'Val_F1': val_f1,
        'Val_MCC': val_mcc,
        'Test_AUC': test_auc,
        'Test_F1': test_f1,
        'Test_MCC': test_mcc,
        'Test_ACC': test_acc,
        'Test_0%': pct_0,
        'Test_1%': pct_1
    })

# === SAVE ALL RESULTS ===
results_df = pd.DataFrame(results)
results_df.to_csv('model_results_per_stock.csv', index=False)
print("\n=== Summary of all stocks ===")
print(results_df.sort_values('Test_AUC', ascending=False))


Training model for: AAL
Test class distribution: 0s=48.28%, 1s=51.72%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[AAL] Test AUC: 0.5674 | F1: 0.6032 | MCC: 0.0510 | ACC: 0.5298
Confusion matrix:
[[ 55  99]
 [ 51 114]]


Training model for: ABBV
Test class distribution: 0s=48.28%, 1s=51.72%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[ABBV] Test AUC: 0.5424 | F1: 0.6819 | MCC: 0.0359 | ACC: 0.5204
Confusion matrix:
[[  2 152]
 [  1 164]]


Training model for: ACN
Test class distribution: 0s=48.28%, 1s=51.72%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[ACN] Test AUC: 0.4994 | F1: 0.6805 | MCC: 0.0027 | ACC: 0.5172
Confusion matrix:
[[  1 153]
 [  1 164]]


Training model for: ADBE
Test class distribution: 0s=47.96%, 1s=52.04%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[ADBE] Test AUC: 0.4966 | F1: 0.6845 | MCC: 0.0000 | ACC: 0.5204
Confusion 

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
[BIIB] Test AUC: 0.5324 | F1: 0.6154 | MCC: 0.0076 | ACC: 0.4828
Confusion matrix:
[[ 22 146]
 [ 19 132]]


Training model for: BSX
Test class distribution: 0s=46.39%, 1s=53.61%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[BSX] Test AUC: 0.5187 | F1: 0.6605 | MCC: 0.0419 | ACC: 0.5392
Confusion matrix:
[[ 29 119]
 [ 28 143]]


Training model for: BX
Test class distribution: 0s=47.34%, 1s=52.66%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[BX] Test AUC: 0.5287 | F1: 0.6437 | MCC: 0.0743 | ACC: 0.5455
Confusion matrix:
[[ 43 108]
 [ 37 131]]


Training model for: CAT
Test class distribution: 0s=49.22%, 1s=50.78%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
[CAT] Test AUC: 0.5065 | F1: 0.6064 | MCC: -0.0215 | ACC: 0.4953
Confusion matrix:
[[ 34 123]
 [ 38 124]]


Training model for: CI
Test class distribu

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[DB] Test AUC: 0.5542 | F1: 0.5266 | MCC: 0.0747 | ACC: 0.5266
Confusion matrix:
[[84 53]
 [98 84]]


Training model for: DE
Test class distribution: 0s=47.96%, 1s=52.04%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[DE] Test AUC: 0.5416 | F1: 0.6845 | MCC: 0.0000 | ACC: 0.5204
Confusion matrix:
[[  0 153]
 [  0 166]]


Training model for: DFS
Test class distribution: 0s=48.59%, 1s=51.41%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[DFS] Test AUC: 0.5075 | F1: 0.6210 | MCC: -0.0877 | ACC: 0.4796
Confusion matrix:
[[ 17 138]
 [ 28 136]]


Training model for: DHI
Test class distribution: 0s=45.45%, 1s=54.55%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[DHI] Test AUC: 0.5412 | F1: 0.6573 | MCC: 0.0393 | ACC: 0.5423
Confusion matrix:
[[ 33 112]
 [ 34 140]]


Training model for: DHR
Test class distribution:

In [7]:
print(results_df['Test_AUC'].mean())
print(results_df['Test_ACC'].mean())
print(results_df['Test_MCC'].mean())

0.5075233283548944
0.5090084525446854
0.011235580624254676
